In [ ]:
from kafka import KafkaConsumer, KafkaProducer
import json

# próg filtrowania
TEMP_THRESHOLD = 30.0

consumer = KafkaConsumer(
    'lab4-topic',
    bootstrap_servers='kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='lab4-router',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

producer = KafkaProducer(
    bootstrap_servers='kafka_streaming_lab:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

print("Routing processor started...")

for msg in consumer:
    data = msg.value
    if data['temperature'] <= TEMP_THRESHOLD:
        target_topic = 'lab4-topic-valid'
    else:
        target_topic = 'lab4-topic-invalid'
    producer.send(target_topic, value=data)
    print(f"Routed to {target_topic}: {data}")
